In [1]:
# Environment Setup
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# Move Kinematics

Every move has a target "entry" and "exit" velocity that we must keep when entering or exiting the move. Within this move, the velocity can be anything. It makes sense to maximize the time spent cruising at a target upper bound on velocity, which we call the "cruise velocity." Spending the most time here means our overall series of paths that we're trying to execute are executing as quickly as possible while respecting the physical acceleration limits we apply to the machine.

## General Case
To follow a trapezoidal velocity profile, we actually have 3 move regions.

When accelerating, we follow
$$
x_a(t) = x_0 + v_0t + \frac{1}{2}at^2
$$
When cruising, we follow:
$$
x_c(t) = x_0 + v_0t
$$
When decelerating, $a$ just flips from positive to negative from our accelerating equation. If $a$ is a constant, we get:
$$
x_d(t) = x_0 + v_0t - \frac{1}{2}at^2
$$



## "Short Move" Case
Some moves are so short, the machine carriage never has enough time to get to the cruise velocity. In that case, we just do our best.

In [67]:
t_i, t_m, t_f, v_i, v_m, v_f, v_c, x_f, t, a = sp.symbols('t_i, t_m, t_f, v_i, v_m, v_f, v_c, x_f, t, a')

x_f_fn = v_i*t_m + sp.Rational(1,2)*a*t_m**2 + v_m*(t_f - t_m) - sp.Rational(1,2)*a*(t_f - t_m)**2 - x_f
v_max_fn = v_i + a*t_m - v_m
v_f_fn = v_m - a*(t_f-t_m) - v_f

# We get two solutions. For positive a, v_i, and v_f (which will always be true), we want the first solution
solns = sp.solve([x_f_fn, v_max_fn, v_f_fn], t_f, v_m, t_m)

t_final, v_max, t_max = solns[0]
givens = {v_f: 1,
          v_i: 1,
          x_f: 1,
          a: 10}
#display(t_final.subs(givens).evalf())
#display(v_max.subs(givens).evalf())
#display(t_max.subs(givens).evalf())

display(v_max)

sqrt(4*a*x_f + 2*v_f**2 + 2*v_i**2)/2